### AMM Optimization, 1st model

In [1]:
using Gurobi, JuMP, CSV, Tables, Ipopt

In [2]:
# With an order amount of token x
function optimize_market_impact(liquidity, order_amount)

    amount_token_x_per_pool = liquidity[:, 1] 
    amount_token_y_per_pool = liquidity[:, 2]

    # Create a new model
    model = Model(Ipopt.Optimizer)

    # Create variables
    @variable(model, x[1:size(liquidity,1)] >= 0)

    # Set objective
    @NLobjective(model, Min, sum(amount_token_y_per_pool[i] / (amount_token_x_per_pool[i]- x[i]) for i in 1:size(liquidity,1)))

    # Add constraint
    @constraint(model, sum(x) == order_amount)
    @constraint(model, x .<= amount_token_x_per_pool)

    # Optimize model
    optimize!(model)

    # Return the optimal solution
    return value.(x)
end

optimize_market_impact (generic function with 1 method)

In [3]:
# With an order amount of token x
function optimize_price_paid(liquidity, order_amount)

    amount_token_x_per_pool = liquidity[:, 1] 
    amount_token_y_per_pool = liquidity[:, 2]

    # Create a new model
    model = Model(Ipopt.Optimizer)

    # Create variables
    @variable(model, x[1:size(liquidity,1)] >= 0)

    # Set objective
    @NLobjective(model, Min, sum((amount_token_y_per_pool[i] * x[i])/ (amount_token_x_per_pool[i]- x[i]) for i in 1:size(liquidity,1)))

    # Add constraint
    @constraint(model, sum(x) == order_amount)
    @constraint(model, x .<= amount_token_x_per_pool)

    # Optimize model
    optimize!(model)

    # Return the optimal solution
    return value.(x)
end

optimize_price_paid (generic function with 1 method)

## EXPERIMENT 1: AMM Optimization, no transaction costs

### Generating random data

We just need to preserve the current magnitude. Also AMMs need to be close to each other in the absence of transaction costs.

In [4]:
ETH_USDT = 1210;

In [5]:
ETH = rand(3000:3500, 12)
USD = ETH .* ETH_USDT
liquidity = [ETH USD];

In [6]:
amount_token_x_per_pool = liquidity[:, 1]
amount_token_y_per_pool = liquidity[:, 2]

model = Model(Gurobi.Optimizer)

#@variable(model, 0 <= x[1:length(liquidity)] <= amount_token_x_per_pool[1:length(liquidity)])
size(liquidity,1)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18


12

#### First experiment: optimization of the market impact

We want to split our large trade into many single trades, such that we minimize the market impact, meaning that the influence of our trade on each AMM is the lowest possible

In [7]:
split_trades = optimize_market_impact(liquidity, 1000)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       12
Number of nonzeros in inequality constraint Jacobian.:       12
Number of nonzeros in Lagrangian Hessian.............:       12

Total number of variables............................:       12
                     variables with only lower bounds:       12
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality co

12-element Vector{Float64}:
   4.2955886680258633e-7
 184.44433838911843
  20.14818817952632
  96.99618294865975
 108.85015518912874
 100.59985254267082
   1.3555358710554317e-7
  30.227097027858466
 115.56712273093163
 160.8275789939863
 182.33948324975034
   1.8325670362299333e-7

### If we have pools with exactly the same magnitude, we have:

In [8]:
ETH_USDT = 1210;
ETH = rand(3000:3000, 12)
USD = ETH .* ETH_USDT
liquidity = [ETH USD];

amount_token_x_per_pool = liquidity[:, 1]
amount_token_y_per_pool = liquidity[:, 2]

split_trades = optimize_market_impact(liquidity, 1000) 

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       12
Number of nonzeros in inequality constraint Jacobian.:       12
Number of nonzeros in Lagrangian Hessian.............:       12

Total number of variables............................:       12
                     variables with only lower bounds:       12
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:       12
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       12

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.4520048e+04 1.00e+03 0.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

12-element Vector{Float64}:
 83.3333333333335
 83.3333333333335
 83.33333333333329
 83.33333333333329
 83.33333333333329
 83.33333333333329
 83.33333333333329
 83.33333333333329
 83.33333333333329
 83.33333333333329
 83.33333333333329
 83.33333333333329

As expected, we split the order into 12 trades, each of 1000/12 = 83.33 ETH. The total amount of ETH we get is 1000, as expected.

 Now we study what happens when we have pools with different magnitudes. We can do the analysis on the sorted list of pools as it does not change anything. and will be easier to interpret

In [9]:
ETH_USDT = 1210;
# arrange for ETH
ETH = rand(3000:3500, 12)
ETH = sort(ETH, rev=false)
USD = ETH .* ETH_USDT
liquidity = [ETH USD];

println("ETH amount", liquidity[:, 1])
println("USDT amount", liquidity[:, 2])
order_amount = 1000
println("Order amount", order_amount)

split_trades = optimize_market_impact(liquidity, order_amount)

ETH amount[3032, 3037, 3050, 3076, 3117, 3186, 3215, 3252, 3279, 3307, 3315, 3400]
USDT amount[3668720, 3674770, 3690500, 3721960, 3771570, 3855060, 3890150, 3934920, 3967590, 4001470, 4011150, 4114000]
Order amount1000
This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       12
Number of nonzeros in inequality constraint Jacobian.:       12
Number of nonzeros in Lagrangian Hessian.............:       12

Total number of variables............................:       12
                     variables with only lower bounds:       12
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:       12
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
    

12-element Vector{Float64}:
   3.2969537168677565
   5.8006976075690995
  12.32003941963829
  25.400026677635907
  46.136604236323485
  81.33330864164445
  96.23547771183837
 115.34058207968648
 129.34631020313523
 143.92715275909373
 148.10355166673102
 192.75929527983612

We can now compare that with the price we would have gotten if we had used the pool with the highest magnitude.

##### Price paid analysis

Here I want to get 1000 ETH, what will be my execution price?

\begin{gather*}
    (x_{eth} - \Delta x_{eth}) (y_{usd} + \Delta y_{usd}) = x_{eth}y_{usd} \\
    \Delta y_{usd} = \frac{y_{usd} \Delta x_{eth}}{x_{eth} - \Delta x_{eth}} \\    
\end{gather*}

Where $x_{eth}$ and $y_{usd}$ are the current balances of the pool ($liq_{i, 1}$ and $liq_{i, 2}$)

Finally, the order execution price one AMM is:

\begin{gather*}
    p_{exec} = \frac{\Delta y_{usd}}{\Delta x_{eth}} = \frac{y_{usd}}{x_{eth} - \Delta x_{eth}}
\end{gather*}

If we split the order over all the AMMs, the overall execution price would be

\begin{gather*}
    p_{exec} = \sum_{i=1}^{n} \frac{y_{i, usd}}{x_{i, eth} - \Delta x_{i, eth}} * \frac{\Delta x_{i, eth}}{order\_size}
\end{gather*}


In [10]:
price = sum((split_trades[i]/order_amount)*liquidity[i, 2] / (liquidity[i, 1] - split_trades[i]) for i in 1:size(liquidity,1)) 
price_exec_baseline_pool = [liquidity[i, 2] / (liquidity[i, 1] - order_amount) for i in 1:size(liquidity,1)]
avg_split = [order_amount / size(liquidity,1) for i in 1:size(liquidity,1)]
price_avg_amm = sum((avg_split[i]/order_amount)*liquidity[i, 2] / (liquidity[i, 1] - avg_split[i]) for i in 1:size(liquidity,1))
println("Price with split trades: ", price)
println("Price without split trades: ", price_exec_baseline_pool)
println("Price with even split on all Amms: ", price_avg_amm)


Price with split trades: 1259.226411609393
Price without split trades: [1805.4724409448818, 1804.0108001963672, 1800.2439024390244, 1792.8516377649325, 1781.5635333018422, 1763.5224153705399, 1756.2753950338601, 1747.3001776198935, 1740.9346204475646, 1734.4906805374947, 1732.6781857451404, 1714.1666666666667]
Price with even split on all Amms: 1242.5174788492407


We see here that our average buying price is actualy higer that the price we would have gotten if we had splitted the order equally over the different AMMs.

##### Market impact analysis

The optimization model that we ran was optimizing the trades to minimize the market impact. That said, we want to evaluate, what is the price post trade in each pool.
It is given by the following formula : 

\begin{gather*}
    p_{post} = \frac{y_{i, usd} + \Delta y_{i, usd}}{x_{i, eth} - \Delta x_{i, eth}}
\end{gather*}

In [11]:
del_y = [liquidity[i, 2]*split_trades[i]/(liquidity[i, 1] - split_trades[i]) for i in 1:size(liquidity,1)]
price_post_trade_per_pool = [(liquidity[i, 2] + del_y[i])/ (liquidity[i, 1] - split_trades[i]) for i in 1:size(liquidity,1)]

# Comparison with an average split
avg_split = [order_amount / size(liquidity,1) for i in 1:size(liquidity,1)]
del_y_avg = [liquidity[i, 2]*avg_split[i]/(liquidity[i, 1] - avg_split[i]) for i in 1:size(liquidity,1)]
price_post_trade_per_pool_avg = [(liquidity[i, 2] + del_y_avg[i])/ (liquidity[i, 1] - avg_split[i]) for i in 1:size(liquidity,1)]

# Comparison with doing the trade on a single pool (each element in the array is "if we did the trade on this pool")
del_y_pool = [liquidity[i, 2]*order_amount/(liquidity[i, 1] - order_amount) for i in 1:size(liquidity,1)]
price_post_trade_per_pool_pool = [(liquidity[i, 2] + del_y_pool[i])/ (liquidity[i, 1] - order_amount) for i in 1:size(liquidity,1)]

println("Simulated price after trade per pool: ", price_post_trade_per_pool)
println("Simulated price after trade per pool if we had done an average split: ", price_post_trade_per_pool_avg)
println("Simulated price after trade per pool if we had done the trade on a single pool (displayed for all the pools): ", price_post_trade_per_pool_pool)

Simulated price after trade per pool: [1212.6357719850093, 1214.6354985335215, 1219.8347935597342, 1230.2333849144704, 1246.631163964793, 1274.2274264228843, 1285.8258555872649, 1300.62385142301, 1311.422388927158, 1322.620872266227, 1325.8204389347427, 1359.8158347916212]
Simulated price after trade per pool if we had done an average split: [1279.3589262630355, 1279.2398814778717, 1278.9322686529479, 1278.3251691059768, 1277.3892458439368, 1275.8707274832227, 1275.252740224792, 1274.480906983836, 1273.9291004715683, 1273.3667428857132, 1273.2078816486649, 1271.5678896997551]
Simulated price after trade per pool if we had done the trade on a single pool (displayed for all the pools): [2693.9923429846863, 2689.6322043182954, 2678.4116597263533, 2656.4603264763646, 2623.114564620615, 2570.257280590366, 2549.176250579621, 2523.1883559591, 2504.8374815478564, 2486.328860224315, 2481.1352854190673, 2428.402777777778]


### Second experiment: optimization of the execution price

We don't care anymore of the market impact, we just want to minimize the execution price. 
Nb: This approach is a simulation and we shouldn't actually do that in real life, because the markets are correlated, and we don't model time here. SO if we have a big market impact on one AMM, there would be arbitrageurs that would take advantage of that.

In [13]:
ETH_USDT = 1210;
# arrange for ETH
ETH = rand(3000:3500, 12)
ETH = sort(ETH, rev=false)
USD = ETH .* ETH_USDT
liquidity = [ETH USD];

println("ETH amount ", liquidity[:, 1])
println("USDT amount ", liquidity[:, 2])
order_amount = 1000
println("Order amount ", order_amount)

split_trades = optimize_price_paid(liquidity, order_amount)

ETH amount[3010, 3024, 3036, 3090, 3191, 3200, 3214, 3241, 3395, 3463, 3475, 3476]
USDT amount[3642100, 3659040, 3673560, 3738900, 3861110, 3872000, 3888940, 3921610, 4107950, 4190230, 4204750, 4205960]
Order amount1000
This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       12
Number of nonzeros in inequality constraint Jacobian.:       12
Number of nonzeros in Lagrangian Hessian.............:       12

Total number of variables............................:       12
                     variables with only lower bounds:       12
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:       12
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
    

12-element Vector{Float64}:
 77.54733994592767
 77.90802524799959
 78.2171840783473
 79.60839881491053
 82.21048563700174
 82.44235475976232
 82.80304006183461
 83.49864743011625
 87.46618575290896
 89.21808579154445
 89.52724462189235
 89.55300785775428

In [14]:
price = sum((split_trades[i]/order_amount)*liquidity[i, 2] / (liquidity[i, 1] - split_trades[i]) for i in 1:size(liquidity,1)) 
price_exec_baseline_pool = [liquidity[i, 2] / (liquidity[i, 1] - order_amount) for i in 1:size(liquidity,1)]
avg_split = [order_amount / size(liquidity,1) for i in 1:size(liquidity,1)]
price_avg_amm = sum((avg_split[i]/order_amount)*liquidity[i, 2] / (liquidity[i, 1] - avg_split[i]) for i in 1:size(liquidity,1))
println("Price with split trades: ", price)
println("Price without split trades: ", price_exec_baseline_pool)
println("Price with even split on all Amms: ", price_avg_amm)

Price with split trades: 1241.9978844373927
Price without split trades: [1811.9900497512438, 1807.8260869565217, 1804.3025540275048, 1788.9473684210527, 1762.259242355089, 1760.0, 1756.5221318879856, 1749.9375278893351, 1715.2192066805846, 1701.270807957775, 1698.888888888889, 1698.691437802908]
Price with even split on all Amms: 1242.0919738521318


In [15]:
del_y = [liquidity[i, 2]*split_trades[i]/(liquidity[i, 1] - split_trades[i]) for i in 1:size(liquidity,1)]
price_post_trade_per_pool = [(liquidity[i, 2] + del_y[i])/ (liquidity[i, 1] - split_trades[i]) for i in 1:size(liquidity,1)]

# Comparison with an average split
avg_split = [order_amount / size(liquidity,1) for i in 1:size(liquidity,1)]
del_y_avg = [liquidity[i, 2]*avg_split[i]/(liquidity[i, 1] - avg_split[i]) for i in 1:size(liquidity,1)]
price_post_trade_per_pool_avg = [(liquidity[i, 2] + del_y_avg[i])/ (liquidity[i, 1] - avg_split[i]) for i in 1:size(liquidity,1)]

# Comparison with doing the trade on a single pool (each element in the array is "if we did the trade on this pool")
del_y_pool = [liquidity[i, 2]*order_amount/(liquidity[i, 1] - order_amount) for i in 1:size(liquidity,1)]
price_post_trade_per_pool_pool = [(liquidity[i, 2] + del_y_pool[i])/ (liquidity[i, 1] - order_amount) for i in 1:size(liquidity,1)]

println("Simulated price after trade per pool: ", price_post_trade_per_pool)
println("Simulated price after trade per pool if we had done an average split: ", price_post_trade_per_pool_avg)
println("Simulated price after trade per pool if we had done the trade on a single pool (displayed for all the pools): ", price_post_trade_per_pool_pool)

Simulated price after trade per pool: [1274.8419379727197, 1274.8419379727177, 1274.8419379727166, 1274.8419379727097, 1274.8419379726977, 1274.8419379726968, 1274.8419379726954, 1274.8419379726924, 1274.8419379726765, 1274.8419379726697, 1274.8419379726688, 1274.8419379726686]
Simulated price after trade per pool if we had done an average split: [1279.8876225216763, 1279.550251553162, 1279.2636577380727, 1278.0026769779893, 1275.763344345932, 1275.5709342560556, 1275.2738570685408, 1274.7084572421038, 1271.6620007907463, 1270.406235562587, 1270.1899196493791, 1270.171963224384]
Simulated price after trade per pool if we had done the trade on a single pool (displayed for all the pools): [2713.477636692161, 2701.0207939508505, 2690.502236752213, 2644.9030470914126, 2566.576559723911, 2560.0, 2549.8925618283583, 2530.811034310279, 2431.3858900545238, 2392.0019520738024, 2385.3086419753085, 2384.7542155908354]


Noticeably, the problem converges towards a solution where we split the order such that the price after trade is the same for all the AMMs.